In [1]:
import pandas as pd
import numpy as np
import os
import time
import scipy.stats as stats
from scipy.optimize import minimize
from utils import load_data, val_loglhood, loglhood, jump_dst, reconstruct_coefs

# Gibbs Sampling

In [18]:
def gibbs_sampling(iters, data_path, K, p, q, mh_iters=1, init_mle=False, n_rows=None, debug=False,\
                   method='normal', X=None, Y0=None, annealing=False, T=None, annealing_n=None,\
                   date_start=None, date_end=None, plant_names=None):
    """
    iters: quantity of samples of A and U.
    data_path: path where data is saved.
    K: number of plants (n_plants in load_data function).
    p: past time to be considered.
    q: jumping distribution for parameters (from scipy.stats).
    mh_iters: haw many samples do with Metropolis Hastings.
    n_rows: how many rows of the data to consider.
    debug: debug mode.
    method: normal - use a jumping distribution from scipy.stats
            personalized - use the jumping distribution personalized by us.
    X, Y0: data to use directly without using load_data function.
    annealing: boolean, use simulated annealing in MH.
    T: simulated annealing decay function.
    """
    if X is None or Y0 is None:
        print('Loading data...')
        Y0, X = load_data(data_path, K, p, resample_rule='10T', n_rows=n_rows,\
                          date_start=date_start, date_end=date_end, plant_names=plant_names)
    print('Y0 shape: {}'.format(Y0.shape))
    print('X shape: {}'.format(X.shape))

    # Theta is the vector of all parameters that will be sampled.
    # A and CovU are reshaped to a 1-D vector theta.
    # Note that this theta change dimensionality when using personalized.
    print('Initializing parameters...')
    theta = init_parameters(K, p, q, Y0, X, debug=debug, method=method)
    
    
    if init_mle:
        print('Calculating MLE...')
        f = lambda theta: -val_loglhood(theta,Y0,X,False, method=method, init_params=False)
        result = minimize(f, theta)
        theta = result.x
        print('Init MLE theta calculated! ({})'.format(-result.fun))
    
    if p == 1 and method == 'personalized':
        A, CovU = reconstruct_coefs(theta, K)
    else:
        A    = np.reshape(theta[:p*K**2],(K*p,K)).swapaxes(0,1)
        CovU = np.reshape(theta[p*K**2:],(K,K)).swapaxes(0,1)
        CovU = np.dot(CovU.T,CovU)
    print(A)
    print(CovU)
    
    if debug:
        print('Parameters intialized!')
    samples = []
    for i in range(iters):
        start_it = time.time()
        print('Iteration {}'.format(i))

        # Loop over all parameters and for each parameter theta[j],
        # do a MH sampling over the distribution of theta[j] given theta[-j].
        for j in range(theta.shape[0]):
            start = time.time()
            mh_samples = metropolis_hastings(theta, j, q, mh_iters, Y0, X, K, debug,\
                                             method=method, annealing=annealing, T=T,\
                                             annealing_n=annealing_n)
            end = time.time()
            # print('Time for sampling theta[{}]: {}'.format(j, end - start))
            # When mh_iters > 1, mh_samples contain mh_iters samples, so a random
            # choice (uniform) is done for selection of the new theta.
            theta[j] = np.random.choice(mh_samples)
        
        lk = val_loglhood(theta,Y0,X,False, method=method, init_params=False)
        print('LK of new theta: {}'.format(lk))

        if p == 1 and method == 'personalized':
            A, CovU = reconstruct_coefs(theta, K)
        else:
            A    = np.reshape(theta[:p*K**2],(K*p,K)).swapaxes(0,1)
            CovU = np.reshape(theta[p*K**2:],(K,K)).swapaxes(0,1)
        samples.append([A.copy(), CovU.copy()])
        end_it = time.time()
        print('Time for iteration {0}: {1:.2f} segs.'.format(i, end_it - start_it))
        remaining_time = ((end_it - start_it) * (iters - (i + 1))) / 60
        print('Estimated remaining time: {0:.2f} mins.'.format(remaining_time))
    print('Finished!')
    return samples


def metropolis_hastings(theta, j, q, iters, Y0, X, K, debug, method='normal',\
                        annealing=False, T=None, annealing_n=None):
    """
    theta: theta vector with all parameters.
    j: theta index of the parameter currently been sampled.
    q: jumping distribution.
    """
    user_std = 1
    samples_mh = [theta[j]] # start sample.
    lk_old = val_loglhood(theta, Y0, X, debug, method=method)
    # print('init lk: {}'.format(lk_old))
    accepted = 0
    rejected = 0
    for t in range(iters):
        lk_new = -np.inf
        c = -1
        while lk_new == -np.inf:
            c += 1
            if method == 'normal':
                x_new = q.rvs(loc=samples_mh[-1], scale=1)
                theta[j] = x_new
            elif method == 'personalized':
                theta, q_eval_new, q_eval_old = jump_dst(theta, j, user_std, K)
            lk_new = val_loglhood(theta, Y0, X, debug, method=method)
            # print('new_lk: {}'.format(lk_new))
        #print('Quantity of -np.infs: {}'.format(c))
        if method == 'normal':
            if annealing and t <= annealing_n:
                logalpha = min([(T(t) ** -1) * (lk_new - lk_old + np.log(q.pdf(samples_mh[-1], loc=x_new) \
                                / q.pdf(x_new, loc=samples_mh[-1]))), 0])
            else:
                logalpha = min([lk_new - lk_old + np.log(q.pdf(samples_mh[-1], loc=x_new) \
                                / q.pdf(x_new, loc=samples_mh[-1])), 0])
        elif method == 'personalized':
            if annealing and t <= annealing_n:
                logalpha = min([(T(t) ** -1) * (lk_new - lk_old + np.log(q_eval_old / q_eval_new)), 0])
            else:
                logalpha = min([lk_new - lk_old + np.log(q_eval_old / q_eval_new), 0])
        alpha = np.exp(logalpha)
        u = stats.uniform.rvs()
        if u < alpha:
            #print('acepted')
            samples_mh.append(theta[j])
            lk_old = lk_new
            accepted += 1
        else:
            #print('rejected')
            rejected += 1
            samples_mh.append(samples_mh[-1])
            theta[j] = samples_mh[-1]

    #print('accepted: {}%%'.format(accepted * 100 / (accepted + rejected)))
    #print(samples_mh)
    return np.array(samples_mh)
        
    
def init_parameters(K, p, q, Y0, X, method='normal', debug=False):
    """
    Initialization of parameters. This functions search a matrix A
    and a matrix CovU that satisfy some conditions that A and CovU
    must satisfy.
    """
    if debug:
        print('Initializing parameters...')
    while True:
        theta = np.zeros(K ** 2 * (p + 1))
        for i in range(theta.shape[0]):
            theta[i] = q.rvs()

        # Force CovU to be positive semidefinite.
        covu = np.reshape(theta[-K**2:], (K, K)).T
        covu = np.dot(covu.T, covu)
        theta[-K**2:] = np.reshape(covu, K**2)
        
        lk = val_loglhood(theta, Y0, X, debug, method=method, init_params=True)
        if debug:
            print('LK = {}'.format(lk))
        if lk != -np.inf:
            print('lk init: {}'.format(lk))
            if p == 1 and method == 'personalized':
                A = np.reshape(theta[:p*K**2],(K*p,K)).swapaxes(0,1)
                eig_valuesA, eig_vecA = np.linalg.eig(A)
                eig_valuesB, eig_vecB = np.linalg.eig(covu)
                theta = np.concatenate((eig_vecA.reshape(-1), eig_vecB.reshape(-1),
                                        eig_valuesA, eig_valuesB))
                if np.all(np.isreal(eig_valuesA)):
                    break
            else:
                break
    return theta

# Test

In [19]:
DATA_PATH = '/home/chrisams/Documents/datasets/data_TAIM/processed/'
date_start = '2011-05'
date_end = '2011-06'
plant_names = [
    'd05b_2010-06-19_2018-03-05.csv',
    'd01_2009-07-12_2018-01-17.csv',
]

q = stats.norm
K = 2
p = 1
iters = 2
debug = False
mh_iters = 10
n_rows = None # Number of rows of the data to load
method = 'normal'
init_mle = False
annealing = True
T0 = 300
TF = 1
annealing_n = 5
X=None
Y0=None
T = lambda t: T0 * ((TF / T0) ** (t / annealing_n))

In [20]:
samples = gibbs_sampling(iters, DATA_PATH, K, p, q, mh_iters=mh_iters, init_mle=init_mle, n_rows=n_rows,\
                         debug=False, method='normal', X=X, Y0=Y0, annealing=annealing, T=T,\
                         annealing_n=annealing_n, date_start=date_start, date_end=date_end,\
                         plant_names=plant_names)

Loading data...
Reading d05b_2010-06-19_2018-03-05.csv...
Reading d01_2009-07-12_2018-01-17.csv...
Y0 shape: (2, 8783)
X shape: (2, 8783)
Initializing parameters...
lk init: -80524.5338115304
[[ 0.01567817 -0.54348559]
 [ 0.54656043 -0.43252028]]
[[ 0.15441977 -0.04256723]
 [-0.04256723  3.14359328]]
Iteration 0
LK of new theta: 2165.284179475292
Time for iteration 0: 38.16
Estimated remaining time: 0.64 mins.
Iteration 1
LK of new theta: 6064.435168425403
Time for iteration 1: 43.31
Estimated remaining time: 0.00 mins.
Finished!


tiempo normal: 108.39511632919312

tiempo personalizado :

In [8]:
import pickle
with open('./samples.pickle', 'wb') as f:
    pickle.dump(samples, f)

In [9]:
samples

[[array([[-0.65870902,  0.10967603],
         [-0.59949607, -0.39017218]]), array([[1.39691405, 1.50945839],
         [0.47145804, 1.28749427]])], [array([[-0.65870902,  0.10967603],
         [-0.64259651, -0.07038095]]), array([[1.52771601, 1.50945839],
         [0.47145804, 1.28749427]])]]

In [10]:
with open('./samples.pickle', 'rb') as f:
    samples_test = pickle.load(f)

In [11]:
samples_test

[[array([[-0.65870902,  0.10967603],
         [-0.59949607, -0.39017218]]), array([[1.39691405, 1.50945839],
         [0.47145804, 1.28749427]])], [array([[-0.65870902,  0.10967603],
         [-0.64259651, -0.07038095]]), array([[1.52771601, 1.50945839],
         [0.47145804, 1.28749427]])]]

In [ ]:
DATA_PATH = '/home/chrisams/Documents/datasets/data_TAIM/processed/'
K = 3
theta_old = np.ones(K*K*2+K*2)
j = 0
user_std = 1
n_rows = 10000
debug = False

In [ ]:
Y0, X = load_data(DATA_PATH, K, 1, resample_rule='10T', n_rows=n_rows)
theta_new, q_eval_new, q_eval_old = jump_dst(theta_old, j, user_std, K)
lk_new = val_loglhood(theta_new, Y0, X, debug)

In [ ]:
print(Y0.shape)
print(X.shape)

In [ ]:
a.pdf(1, loc=2, scale=2)

In [ ]:
b.pdf(1, loc=2, scale=2)

In [ ]:
A = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
print(A)
U = np.array([[7, 8, 9], [10, 11, 12], [13, 14, 15]])
print(U)

In [ ]:
Av = A.T.reshape(-1)
print(Av)
Uv = U.T.reshape(-1)
print(Uv)
theta = np.concatenate([Av, Uv])
print(theta)

In [ ]:
Kv = 3
pv = 1
A = np.reshape(theta[:pv*Kv**2],(Kv*pv,Kv)).swapaxes(0,1)
CovU = np.reshape(theta[pv*Kv**2:],(Kv,Kv)).swapaxes(0,1)

In [ ]:
print(A)
print(CovU)

In [ ]:
K = 3
a = np.array([[1, 2, 3], [2, 1, 4], [3, 4, 5]])
b = np.array([[1, 2, 16], [2, 1, 4], [16, 4, 5]])
print(a)
print(b)

In [ ]:
eig_valuesA, eig_vecA = np.linalg.eig(a)
eig_valuesB, eig_vecB = np.linalg.eig(b)
theta = np.concatenate((eig_vecA.reshape(-1),eig_vecB.reshape(-1),
                        eig_valuesA,eig_valuesB))

In [ ]:
samp_vecA = np.reshape(theta[:(K*K)],(K,K))
samp_vecU = np.reshape(theta[(K*K):(K*K*2)],(K,K))
samp_valA = np.diag(theta[(K*K*2):(K*K*2+K)])
samp_valU = np.diag(theta[(K*K*2+K):(K*K*2+K*2)])

A = samp_vecA @ samp_valA @np.linalg.inv(samp_vecA)
U = samp_vecU @ samp_valU @np.linalg.inv(samp_vecU)

In [ ]:
print(A)
print(U)